# Imports

we want to import openCV, mediapipe, numpy

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [14]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
  
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 


In [17]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('./training_data') 

# Actions that we try to detect
actions = np.array(['jab', 'cross', 'guard', 'lead_hook', 'lead_uppercut'])

# Thirty videos worth of data
no_sequences = 100

# Videos are going to be 30 frames in length
sequence_length = 25

# Folder start
for action in actions: 
    # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [21]:

frameCount = 0
jumpFrame = -1
pendingSave = False
clipStartFrame = 0
frames = []

action = input("Enter Action")
clipNum = int(input("Enter current recording num"))
# Set mediapipe model 

cap = cv2.VideoCapture('./videos/' + action + '.MOV')

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        if jumpFrame >= 0:
            jumpFrame -= 1

        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        frame = []
        if jumpFrame >= 0:
            if results.pose_landmarks:
                for res in results.pose_landmarks.landmark:
                    frame.append(res.x)
                    frame.append(res.y)
                    frame.append(res.z)
                    frame.append(res.visibility)
            else:
                for i in range(132):
                    frame.append(0)

            frames.append(frame)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen

        if pendingSave:
            cv2.putText(image, text= str("Frames: " + str(clipStartFrame) + " - " + str(frameCount) + ". Press c to save these frames to " + str(clipNum) + ".npy"), org=(700, 800),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,255),
                thickness=2, lineType=cv2.LINE_AA)
            
            cv2.putText(image, text= str("Press x to discard these frames"), org=(700, 850),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,255),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.putText(image, text= str("Action #" + str(clipNum) + ". Saving to " + str(clipNum) + ".npy"), org=(700, 1000),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,0,0),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.putText(image, text= str(frameCount), org=(500, 300),
                fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,0),
                thickness=2, lineType=cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        
        wait = 10 if jumpFrame >= 0 else 0
        
        key = cv2.waitKey(wait)
        if key == ord('q'):
            break

        if pendingSave and key == ord('c'):
            pendingSave = False
            
            print("Saved")
            print("---------------")

            #For loop that goes through each frame and writes a file to the corresponding action number
            for i in range(0, len(frames)):
                frame = np.array(frames[i])
                print(frame)
                np.save( "./training_data/" + action + "/"+ str(clipNum) + "/" + str(i+1) + ".npy", frame)

            clipNum += 1
            frames = []
            continue

        if pendingSave and key == ord('x'):
            pendingSave = False
            print("Discarded")
            print("---------------")
            frames = []
            frameCount = clipStartFrame
            cap.set(cv2.CAP_PROP_POS_FRAMES, clipStartFrame)
            continue

        if not pendingSave and jumpFrame < 0:
            if key == ord('p'):
                frameCount -= 1
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
                continue
            if key == ord('d'):
                frameCount += 100
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
                continue
            if key == ord('a'):
                frameCount -= 100
                cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)
                continue
            if key == ord('s'):
                clipStartFrame = frameCount
                jumpFrame = 25
                pendingSave = True
                continue
            else:
                frameCount += 1
                continue

            # Quit when 'q' is pressed

        # print("pog")
        frameCount += 1

    cap.release()
    cv2.destroyAllWindows()


# print(frames)

Saved
---------------
[ 0.54427934  0.06459633 -0.24645191  0.99963284  0.54868275  0.05216716
 -0.23962764  0.99932849  0.55120283  0.05182733 -0.23963055  0.99942529
  0.55341905  0.05133231 -0.23967537  0.99955952  0.539882    0.0504851
 -0.23514079  0.9993149   0.53680354  0.04905052 -0.23515752  0.9992246
  0.53407031  0.04767296 -0.23515971  0.9992379   0.55701816  0.05122272
 -0.16182597  0.99962342  0.53031909  0.04645362 -0.14244993  0.99901432
  0.54875541  0.07658844 -0.21589252  0.99981982  0.53867275  0.07410812
 -0.20935635  0.99972457  0.57664663  0.11273848 -0.12398905  0.99996191
  0.51296633  0.10779488 -0.08835649  0.99875516  0.59856641  0.19546491
 -0.25879151  0.9951812   0.48830417  0.19060341 -0.21491025  0.53525907
  0.57998168  0.09599705 -0.40177342  0.98988962  0.50783855  0.09199829
 -0.37678546  0.53958172  0.57453215  0.07147811 -0.43261486  0.97253001
  0.51316154  0.06852489 -0.40747464  0.52407742  0.5723232   0.06679717
 -0.41403466  0.9679513   0.515